Oulu_NLPTM_TwitterBrexit

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
import os
import tweepy as tw
import pandas as pd

import re
import string

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.stem import PorterStemmer

import sys
sys.path.insert(0, '../config')

from twitter_dev_access import *


# Global Parameters
stop_words = set(stopwords.words('english'))

In [ ]:
# access twitter
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)


In [ ]:
conservatives = ['@BorisJohnson', '@theresa_may']
labour = ['@Keir_Starmer']

In [ ]:
def collect_tweets(twitterusers):
    tweet_list = []

    for twitter_name in twitterusers:
        print('collecting tweets from ' + twitter_name)
        
        # get tweets for user, exlude retweets and get full tweet content
        pages = tw.Cursor(api.user_timeline, screen_name=twitter_name, include_rts=False, tweet_mode="extended", count=5).pages(1)
        
        # get tweets from pages
        tweet_texts = [tweet.full_text for page in pages for tweet in page]

        print('Collected ' + str(len(tweet_texts)) + ' tweets from ' + twitter_name)

        # only add newest 100 tweets to tweet list
        tweet_list.extend(tweet_texts[:1000])
    
    print('All tweets collected for this party: '+ str(len(tweet_list)))
    
    return tweet_list

In [ ]:
def clean(tweet):
    # from https://www.pluralsight.com/guides/building-a-twitter-sentiment-analysis-in-python
    tweet.lower()
    # Remove urls
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    # Remove user @ references and '#' from tweet
    tweet = re.sub(r'\@\w+|\#','', tweet)
    # Remove punctuations
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords
    tweet_tokens = word_tokenize(tweet)
    filtered_words = [w for w in tweet_tokens if not w in stop_words]
    
    # use stemmer
    ps = PorterStemmer()
    stemmed_words = [ps.stem(w) for w in filtered_words]
    
    return " ".join(stemmed_words)
    

In [ ]:
conservatives_tweets = collect_tweets(conservatives)

In [ ]:
cleaned_conservative = [clean(tweet) for tweet in conservatives_tweets]

In [ ]:
print(conservatives_tweets)
print(cleaned_conservative)

In [ ]:
conservative_tweets = []

for twitter_name in conservatives:
    print('collecting tweets from ' + twitter_name)
    pages = tw.Cursor(api.user_timeline, screen_name=twitter_name, include_rts=False, tweet_mode="extended", count=200).pages(12)
    
    tweet_texts = [clean(tweet.full_text) for page in pages for tweet in page]
    
    print('Collected ' + str(len(tweet_texts)) + ' tweets from ' + twitter_name)
    
    conservative_tweets.extend(tweet_texts)
    
print('All tweets collected from conservative politicians: '+ str(len(conservative_tweets)))

In [ ]:
labour_tweets = []

for twitter_name in labour:
    print('collecting tweets from ' + twitter_name)
    pages = tw.Cursor(api.user_timeline, screen_name=twitter_name, tweet_mode="extended", count=200).pages(5)
    
    tweet_texts = [tweet.full_text for page in pages for tweet in page]
    
    print('Collected ' + str(len(tweet_texts)) + ' tweets from ' + twitter_name)
    
    labour_tweets.extend(tweet_texts)
    
print('All tweets collected from labour politicians: '+ str(len(labour_tweets)))

In [ ]:
for tweet in conservative_tweets:
    print(tweet)